In [8]:
import os
import time
import tensorflow.compat.v1 as tf
import pickle

tf.enable_eager_execution() # No need for session to be created. Function instances are run immediately. 

from waymo_open_dataset import dataset_pb2 as open_dataset
from google.cloud import storage

import concurrent.futures as concurr

# CONFIG
project = "Waymo3DObjectDetection"
bucket_name = 'waymo_open_dataset_v_1_2_0_individual_files'
suffix = '.tfrecord'
data_destination = os.getcwd() + "/data/"
download_batch_size = 1

def download_blob(blob, c):
    """
    blob = single file name
    c = file counter
    """
    fname = f"{data_destination}blob_{c}{suffix}"
    blob.download_to_filename(fname)
    return fname

def _strip_frame(frame, idx, blob_idx):
    """Strip frame from garbage such as LIDAR data"""
    
    cam_dict = {}
    for i, camera in enumerate(["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]):
        cam_dict[camera] = {}
        cam_dict[camera]['image'] = frame.images[i].image
        cam_dict[camera]['velocity'] = frame.images[i].velocity
        cam_dict[camera]['labels'] = frame.camera_labels[i]
        
    cam_dict['context']={'stats':frame.context.stats, 
                       'name': frame.context.name, 
                       'blob_idx':blob_idx,
                       'time_frame_idx':idx}
    return cam_dict

def save_frames(frames, blob_idx, dataset='training'):
    """Save frames into pickle format. To preprocess later"""
    with open(f'{data_destination}{dataset}/blob_{blob_idx}.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(frames, f, pickle.HIGHEST_PROTOCOL)
    return None

def load_frame(frame_idx, blob_idx, dataset='training'):
    with open(f'{data_destination}{dataset}/blob_{blob_idx}.pickle', 'rb') as f:
        # Load the 'data' dictionary using the highest protocol available.
        return pickle.load(f, pickle.HIGHEST_PROTOCOL)


# Retrieve frames from selected files to download
def get_and_strip_frames_from_one_blob(downloaded_blob, blob_idx):
    # Load into tf record dataset
    dataset = tf.data.TFRecordDataset(downloaded_blob, compression_type='')
    frames = []
    for idx, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        # Function to strip away LIDAR and other garbage from frame
        frame = _strip_frame(frame, idx, blob_idx)
        frames+=frame
    return frames

def download_process_save_1_blob(blob, blob_idx, dataset='training'):
    """Like dem descriptive func names eh?"""

    print(f"Downloading blob_{blob_idx}")
    blob_fname = download_blob(blob, blob_idx)
    print(f'Blob_{blob_idx} downloaded')
    
    print("Getting and stripping all frames from blob_{blob_idx}")
    frames = get_and_strip_frames_from_one_blob(blob_fname, blob_idx)
    print("Frames processed")
    
    print(f"Saving frames to {dataset}/blob_{blob_idx}.pickle")
    save_frames(frames, blob_idx, dataset)

    print(f'No longer need tfrecord blob_{blob_idx}. Deleting now.')
    os.remove(f'data/blob_{blob_idx}.tfrecord')

    return f"blob_{blob_idx}"
          

In [2]:
# Initialise a client
storage_client = storage.Client(project= project) #storage.Client(project= "Waymo3DObjectDetection", credentials=credentials)
# Create a bucket object for our bucket
bucket = storage_client.get_bucket(bucket_name)
# Get blob files in bucket
blobs = [blob for blob in storage_client.list_blobs(bucket_name, prefix='training/')]

n_blobs = len(blobs) # Number of blobs in the training dataset
print(f'Total number of blobs is {n_blobs}')


/usr/local/lib/python3.6/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Total number of blobs is 798


In [3]:
# TRAINING
start = time.time()
downloaded_blobs = []

thread_iterable = ((blob,blob_idx, 'training') for blob_idx, blob in enumerate(blobs))

with concurr.ThreadPoolExecutor(max_workers = 2) as executor:

    results = executor.map(lambda args: download_process_save_1_blob(*args), thread_iterable)
    for r in results:
        print(f'\n Time elapsed {time.time() - start}')
        downloaded_blobs.append(r)
        
end = time.time()
print(f'Total time taken {end - start}')


Blob_1 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_1.pickle
No longer need tfrecord blob_1. Deleting now.
Blob_0 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_0.pickle
No longer need tfrecord blob_0. Deleting now.

 Time elapsed 21.635506629943848

 Time elapsed 21.636046171188354
Blob_2 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_2.pickle
No longer need tfrecord blob_2. Deleting now.
 Time elapsed 34.9662230014801

Blob_3 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_3.pickle
No longer need tfrecord blob_3. Deleting now.
 Time elapsed 42.89931106567383

Blob_4 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_4.pickle
No longer need tfrecord blob_4. Deleting no

No longer need tfrecord blob_35. Deleting now.
 Time elapsed 353.2066431045532

Blob_37 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_37.pickle
No longer need tfrecord blob_37. Deleting now.
Blob_36 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_36.pickle
No longer need tfrecord blob_36. Deleting now.

 Time elapsed 371.8528027534485

 Time elapsed 371.85305070877075
Blob_38 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_38.pickle
No longer need tfrecord blob_38. Deleting now.
 Time elapsed 386.27673602104187

Blob_39 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_39.pickle
No longer need tfrecord blob_39. Deleting now.
 Time elapsed 392.8003888130188

Blob_40 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frame

Frames processed
Saving frames to pickled/blob_71.pickle
Blob_72 downloaded
Getting and stripping all frames from blob_{blob_idx}
No longer need tfrecord blob_71. Deleting now.
 Time elapsed 701.1238179206848

Frames processed
Saving frames to pickled/blob_72.pickle
No longer need tfrecord blob_72. Deleting now.
 Time elapsed 702.5690317153931

Blob_73 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_73.pickle
No longer need tfrecord blob_73. Deleting now.
Blob_74 downloaded
Getting and stripping all frames from blob_{blob_idx}
 Time elapsed 722.7521514892578

Frames processed
Saving frames to pickled/blob_74.pickle
No longer need tfrecord blob_74. Deleting now.

 Time elapsed 725.1443548202515
Blob_75 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_75.pickle
No longer need tfrecord blob_75. Deleting now.
 Time elapsed 738.2994706630707

Blob_76 downloaded
Gett

Blob_106 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_106.pickle
No longer need tfrecord blob_106. Deleting now.

 Time elapsed 1035.932873249054

 Time elapsed 1035.9331591129303
Blob_108 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_108.pickle
No longer need tfrecord blob_108. Deleting now.

 Time elapsed 1049.6813387870789
Blob_109 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_109.pickle
No longer need tfrecord blob_109. Deleting now.

 Time elapsed 1055.6772420406342
Blob_110 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_110.pickle
No longer need tfrecord blob_110. Deleting now.

 Time elapsed 1068.7522649765015
Blob_111 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pic

 Time elapsed 1388.0317614078522

Blob_142 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_142.pickle
No longer need tfrecord blob_142. Deleting now.

 Time elapsed 1398.103273153305
Blob_143 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_143.pickle
No longer need tfrecord blob_143. Deleting now.
 Time elapsed 1409.0107951164246

Blob_144 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_144.pickle
No longer need tfrecord blob_144. Deleting now.

 Time elapsed 1420.8386478424072
Blob_145 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_145.pickle
No longer need tfrecord blob_145. Deleting now.

 Time elapsed 1424.007227897644
Blob_146 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pick

...

Blob_634 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_634.pickle
No longer need tfrecord blob_633. Deleting now.
 Time elapsed 6492.950666189194

No longer need tfrecord blob_634. Deleting now.
 Time elapsed 6493.174783945084

Blob_635 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_635.pickle
Blob_636 downloaded
Getting and stripping all frames from blob_{blob_idx}
No longer need tfrecord blob_635. Deleting now.

 Time elapsed 6511.977565526962
Frames processed
Saving frames to pickled/blob_636.pickle
No longer need tfrecord blob_636. Deleting now.
 Time elapsed 6513.0532648563385

Blob_637 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_637.pickle
No longer need tfrecord blob_637. Deleting now.
 Time elapsed 6529.01712846756

Blob_638 downloaded
Getting and stripping all frames from blob_{blo


 Time elapsed 6847.681392908096
Blob_669 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_669.pickle
No longer need tfrecord blob_669. Deleting now.

 Time elapsed 6865.31489276886
Blob_670 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_670.pickle
No longer need tfrecord blob_670. Deleting now.
 Time elapsed 6870.79665184021

Blob_671 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_671.pickle
No longer need tfrecord blob_671. Deleting now.
 Time elapsed 6887.029671430588

Blob_672 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_672.pickle
No longer need tfrecord blob_672. Deleting now.
 Time elapsed 6891.358879327774

Blob_673 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/b

Blob_704 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_704.pickle
No longer need tfrecord blob_704. Deleting now.
 Time elapsed 7250.041125535965

Blob_705 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_705.pickle
No longer need tfrecord blob_705. Deleting now.
 Time elapsed 7257.647850751877

Blob_706 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_706.pickle
No longer need tfrecord blob_706. Deleting now.
 Time elapsed 7267.100435256958

Blob_707 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_707.pickle
No longer need tfrecord blob_707. Deleting now.
 Time elapsed 7277.179228782654

Blob_708 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_708.pickle
No longer need t

Frames processed
Saving frames to pickled/blob_739.pickle
No longer need tfrecord blob_739. Deleting now.
 Time elapsed 7616.816335201263

Blob_740 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_740.pickle
No longer need tfrecord blob_740. Deleting now.

 Time elapsed 7623.762544393539
Blob_741 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_741.pickle
No longer need tfrecord blob_741. Deleting now.
 Time elapsed 7638.406848192215

Blob_742 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_742.pickle
No longer need tfrecord blob_742. Deleting now.
 Time elapsed 7652.7014100551605

Blob_743 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_743.pickle
No longer need tfrecord blob_743. Deleting now.
 Time elapsed 7664.234098911285

Blob_744

No longer need tfrecord blob_774. Deleting now.
 Time elapsed 8006.311390399933

Blob_775 downloaded
Getting and stripping all frames from blob_{blob_idx}
Blob_776 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_775.pickle
No longer need tfrecord blob_775. Deleting now.
Frames processed
Saving frames to pickled/blob_776.pickle
 Time elapsed 8023.671188592911

No longer need tfrecord blob_776. Deleting now.

 Time elapsed 8025.412765741348
Blob_778 downloaded
Getting and stripping all frames from blob_{blob_idx}
Blob_777 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to pickled/blob_778.pickle
Frames processed
Saving frames to pickled/blob_777.pickle
No longer need tfrecord blob_778. Deleting now.
No longer need tfrecord blob_777. Deleting now.
 Time elapsed 8044.497521400452


 Time elapsed 8044.497733831406
Blob_779 downloaded
Getting and stripping all frames from blob_{blo

In [10]:
# VALIDATION
# val blobs
blobs = [blob for blob in storage_client.list_blobs(bucket_name, prefix='validation/')]

start = time.time()
downloaded_blobs = []

thread_iterable = ((blob,blob_idx, 'validation') for blob_idx, blob in enumerate(blobs))

with concurr.ThreadPoolExecutor(max_workers = 2) as executor:

    results = executor.map(lambda args: download_process_save_1_blob(*args), thread_iterable)
    for r in results:
        print(f'\n Time elapsed {time.time() - start}')
        downloaded_blobs.append(r)
        
end = time.time()
print(f'Total time taken {end - start}')


Blob_0 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_0.pickle
No longer need tfrecord blob_0. Deleting now.
 Time elapsed 23.018300533294678

Blob_1 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_1.pickle
No longer need tfrecord blob_1. Deleting now.
 Time elapsed 26.122740983963013

Blob_2 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_2.pickle
No longer need tfrecord blob_2. Deleting now.
 Time elapsed 40.86631917953491

Blob_3 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_3.pickle
No longer need tfrecord blob_3. Deleting now.
 Time elapsed 45.47329330444336

Blob_4 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_4.pickle
No longer need tfrecord blo

Frames processed
Saving frames to validation/blob_35.pickle
No longer need tfrecord blob_35. Deleting now.
 Time elapsed 354.403639793396

Blob_36 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_36.pickle
Blob_37 downloaded
Getting and stripping all frames from blob_{blob_idx}
No longer need tfrecord blob_36. Deleting now.
Frames processed
Saving frames to validation/blob_37.pickle

 Time elapsed 367.1398985385895
No longer need tfrecord blob_37. Deleting now.
 Time elapsed 368.04579281806946

Blob_38 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_38.pickle
No longer need tfrecord blob_38. Deleting now.
 Time elapsed 384.12442922592163

Blob_39 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_39.pickle
No longer need tfrecord blob_39. Deleting now.
 Time elapsed 390.8840637207031

Blob_40

Frames processed
Saving frames to validation/blob_71.pickle
No longer need tfrecord blob_71. Deleting now.
Frames processed
Saving frames to validation/blob_70.pickle
No longer need tfrecord blob_70. Deleting now.
 Time elapsed 686.4671380519867


 Time elapsed 686.4674246311188
Blob_72 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_72.pickle
No longer need tfrecord blob_72. Deleting now.
 Time elapsed 704.516930103302

Blob_73 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_73.pickle
No longer need tfrecord blob_73. Deleting now.
 Time elapsed 707.7766473293304

Blob_74 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_74.pickle
No longer need tfrecord blob_74. Deleting now.
 Time elapsed 723.4545488357544

Blob_75 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames pr

 Time elapsed 1080.4528682231903

Blob_106 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_106.pickle
No longer need tfrecord blob_106. Deleting now.

 Time elapsed 1084.135436296463
Blob_107 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_107.pickle
No longer need tfrecord blob_107. Deleting now.
 Time elapsed 1096.1815676689148

Blob_108 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_108.pickle
No longer need tfrecord blob_108. Deleting now.
 Time elapsed 1103.60533618927

Blob_109 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_109.pickle
No longer need tfrecord blob_109. Deleting now.
 Time elapsed 1115.1365110874176

Blob_110 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving fra

 Time elapsed 1474.9551949501038

No longer need tfrecord blob_140. Deleting now.
 Time elapsed 1475.4531815052032

Blob_142 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_142.pickle
Blob_141 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_141.pickle
No longer need tfrecord blob_142. Deleting now.
No longer need tfrecord blob_141. Deleting now.
 Time elapsed 1497.1971499919891

 Time elapsed 1497.1972742080688

Blob_143 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_143.pickle
No longer need tfrecord blob_143. Deleting now.
 Time elapsed 1515.024617433548

Blob_144 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_144.pickle
No longer need tfrecord blob_144. Deleting now.
 Time elapsed 1518.92560172081

Blob_146 downloaded

Blob_175 downloaded
Getting and stripping all frames from blob_{blob_idx}
Blob_176 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_175.pickle
Frames processed
Saving frames to validation/blob_176.pickle
No longer need tfrecord blob_175. Deleting now.
No longer need tfrecord blob_176. Deleting now.
 Time elapsed 1925.2798998355865

 Time elapsed 1925.2800557613373

Blob_177 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_177.pickle
No longer need tfrecord blob_177. Deleting now.
 Time elapsed 1931.5904400348663

Blob_178 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_178.pickle
No longer need tfrecord blob_178. Deleting now.
 Time elapsed 1943.3247861862183

Blob_179 downloaded
Getting and stripping all frames from blob_{blob_idx}
Frames processed
Saving frames to validation/blob_179.pick